In [ ]:
# Test interpolation to connect 2 phrases

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from generate import generate_z, generate_music

In [ ]:
# Function to interpolate between two latent vectors
def interpolate_latents(z1, z2, steps=10):
    """Interpolate between two latent vectors z1 and z2."""
    interpolated_z = []
    for t in np.linspace(0, 1, steps):
        z_t = (1 - t) * z1 + t * z2
        interpolated_z.append(z_t)
    return torch.stack(interpolated_z)

In [ ]:
# Get latent vector for start phrase
z1 = generate_z(1, d_model, device)

# Get latent vector for end phrase
z2 = generate_z(1, d_model, device)

# Interpolate between 2 latent vectors
zi = interpolate_latents(z1, z2, steps=1)

# Generate music for interpolated vector
mtp, s_tensor = generate_music(vae, zi, s_cond=None, s_tensor_cond=None)

# Combine all 3 